In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 2s 0us/step


In [14]:
import os
import glob
import pandas as pd
dn = os.path.split(dataset)[0]

def read_file(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(dn):
    posfn = glob.glob("{}/pos/*".format(dn))
    negfn = glob.glob("{}/neg/*".format(dn))
    postarget = [1] * len(posfn)
    negtarget = [0] * len(negfn)
    df = pd.DataFrame({
        "fname":posfn + negfn,
        "target":postarget + negtarget
    })
    df["content"] = df["fname"].apply(read_file)
    return df

train_df = get_data(os.path.join(dn, "aclImdb", "train"))
train_df

,fname,target,content
0,/root/.keras/datasets/aclImdb/train/pos/4971_7...,1,This is an absurdist dark comedy from Belgium....
1,/root/.keras/datasets/aclImdb/train/pos/9502_8...,1,I liked this movie very much. Although this mo...
2,/root/.keras/datasets/aclImdb/train/pos/10650_...,1,"I rented this movie this past weekend, cranked..."
3,/root/.keras/datasets/aclImdb/train/pos/1683_7...,1,I saw this film as it was the second feature o...
4,/root/.keras/datasets/aclImdb/train/pos/5784_8...,1,I've noticed that a lot of people who post on ...
...,...,...,...
24995,/root/.keras/datasets/aclImdb/train/neg/6299_1...,0,This inept adaptation of arguably one of Marti...
24996,/root/.keras/datasets/aclImdb/train/neg/2270_1...,0,The only thing that kept me from vomiting afte...
24997,/root/.keras/datasets/aclImdb/train/neg/3347_3...,0,I am not sure why I like Dolph Lundgren. I gue...
24998,/root/.keras/datasets/aclImdb/train/neg/9728_3...,0,Tobe Hooper has made great movies so I was cer...


In [15]:
test_df = get_data(os.path.join(dn, "aclImdb", "test"))
test_df

,fname,target,content
0,/root/.keras/datasets/aclImdb/test/pos/1010_9.txt,1,Hehehe. This was one of the best funny road mo...
1,/root/.keras/datasets/aclImdb/test/pos/10037_7...,1,"This movie is good for entertainment purposes,..."
2,/root/.keras/datasets/aclImdb/test/pos/10135_8...,1,Corniness Warning. As many fellow IMDb users a...
3,/root/.keras/datasets/aclImdb/test/pos/11555_7...,1,This movie starts off somewhat slowly and gets...
4,/root/.keras/datasets/aclImdb/test/pos/3466_10...,1,I got some free tickets via the Times to see t...
...,...,...,...
24995,/root/.keras/datasets/aclImdb/test/neg/5389_4.txt,0,"It was definitely worth viewing, I don't regre..."
24996,/root/.keras/datasets/aclImdb/test/neg/8021_1.txt,0,I can confidently say that this is the worst f...
24997,/root/.keras/datasets/aclImdb/test/neg/88_3.txt,0,"Wow, what an overrated movie this turned out t..."
24998,/root/.keras/datasets/aclImdb/test/neg/2270_1.txt,0,The comparisons between the 1995 version and t...
